In [75]:
access_token = "eyJhbGciOiJSUzI1NiIsImtpZCI6ImZlbmNlX2tleV9rZXktMDEiLCJ0eXAiOiJKV1QifQ.eyJwdXIiOiJhY2Nlc3MiLCJpc3MiOiJodHRwczovL3FhLW1pZHJjLnBsYW54LXBsYS5uZXQvdXNlciIsImF1ZCI6WyJodHRwczovL3FhLW1pZHJjLnBsYW54LXBsYS5uZXQvdXNlciIsIm9wZW5pZCIsInVzZXIiLCJmZW5jZSIsImRhdGEiLCJjcmVkZW50aWFscyIsImdvb2dsZV9zZXJ2aWNlX2FjY291bnQiXSwiaWF0IjoxNzQ1NTkzOTY0LCJleHAiOjE3NDU1OTc1NjQsImp0aSI6Ijc4YTQ2MGRhLTBmYzUtNGU2Yi1hODNjLTczN2M0YzdlNzcxYSIsInNjb3BlIjpbIm9wZW5pZCIsInVzZXIiLCJmZW5jZSIsImRhdGEiLCJjcmVkZW50aWFscyIsImdvb2dsZV9zZXJ2aWNlX2FjY291bnQiXSwiY29udGV4dCI6eyJ1c2VyIjp7Im5hbWUiOiJtYWluQGV4YW1wbGUub3JnIiwiaXNfYWRtaW4iOnRydWUsImdvb2dsZSI6eyJwcm94eV9ncm91cCI6bnVsbH19fSwiYXpwIjoiIiwic3ViIjoiMzEzIn0.Z94Ebc2mEMg4prCO01qYmny1Rljg1TbCCLxKy98x_9o7tf-erDQL6QVMlMpIKxcYbUABD1nKZlMAEmWMmQFbVyGFlTOYHpD-E2Qalz74BVAm3Je-Pdi_n37X1LpCB3PPO0boPk9aJju9jJmEMOmjI9Kz8jDLMKLy0vvZnTRVxOixReqkt8plUIV6yAP4MzYh53eI1Uko7zQIlxdC7JRU5Mu0Ht8nQeiKcVLcrR92InxH4mZF4vzfPftQoFc2-VV5A0KB-NTBphIqh22LFijS_Sakt4Dx_GRvewxWtrCT4-N-rXtDMGRZUCTO-fJgBtPaR6DXiV5mZvxj0AuL-tX7BQ"


In [68]:
import nextflow
import os
os.environ["GEN3_TOKEN"] = access_token
execution = nextflow.run("./main.nf", configs = ["./nextflow.config"])

In [76]:
import boto3
from botocore.config import Config


def get_s3_objects(
    folder_path,
    method="list",
    access_token=access_token,
):
    client = boto3.client(
        service_name="s3",
        aws_access_key_id=access_token,
        aws_secret_access_key="N/A",
        endpoint_url="https://qa-midrc.planx-pla.net/workflows/s3",
        config=Config(region_name="us-east-1"),
    )
    bucket, _, key = folder_path.partition("/")
    if method == "list":
        # list all objects in the bucket
        print(f"Listing objects in bucket: {bucket} with prefix: {key}")
        response = client.list_objects_v2(Bucket=bucket, Prefix=key)
    elif method == "get":
        # get a specific object from the bucket
        pass
    return response

In [70]:
from datetime import datetime
print(f"{execution.status=}")
with open(f"execution_{datetime.now()}.log", "w") as log_file:
    log_file.write(execution.log)

execution.status='ERROR'


In [ ]:
import re
def nextflow_parse_completed_line(line):
    """Parses a line from the log file that indicates a process has completed,
    to get its xx/yyyyyy identifier, finish time, return code, and status.

    :param str line: a line from the log file.
    :rtype: ``dict``"""

    log_pattern = (
        r"(?P<timestamp>\w{3}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3}) .*?"
        r"Task completed > TaskHandler\[.*?"
        r"name: (?P<name>.+); status: (?P<status>-?\w+); "
        r"exit: (?P<exit_code>-?\d+); "
        r".*?workDir: (?P<workDirProtocol>.+):\/\/(?P<workDir>.+)]"
    )
    match = re.match(log_pattern, line)
    if not match: return {}
    process_name = match.group("name")
    workDir = match.group("workDir")
    exit_code = match.group("exit_code")
    status = match.group("status") or "-"
    if exit_code != "0": status = "FAILED"
    return {
        "process_name": process_name,
        "workDir": workDir,
        "exit_code": exit_code,
        "status": status,
    }

In [83]:

def get_lines_with_process_info(log):
    for line in log.split("\n"):
        if "Task completed > TaskHandler" in line:
            yield line
s3_contents = {}
log_file_content = ""
with open(".nextflow.log", "r") as log_file:
    log_file_content = log_file.read()
for line in get_lines_with_process_info(log_file_content):
    parsed_line = nextflow_parse_completed_line(line)
    print(parsed_line)
    s3_contents[parsed_line["process_name"]] = get_s3_objects(folder_path=parsed_line['workDir'].replace("s3://", ""), method = "list", access_token=access_token)


Work Dir protocol: s3
{'process_name': 'extract_metadata (2)', 'workDir': 'gen3wf-qa-midrc-planx-pla-net-313/ga4gh-tes/b7/34a4f2e25237ce38d536961cf17527', 'exit_code': '0', 'status': 'COMPLETED'}
Listing objects in bucket: gen3wf-qa-midrc-planx-pla-net-313 with prefix: ga4gh-tes/b7/34a4f2e25237ce38d536961cf17527
Work Dir protocol: s3
{'process_name': 'extract_metadata (1)', 'workDir': 'gen3wf-qa-midrc-planx-pla-net-313/ga4gh-tes/ba/5527daa7d08eba26c89e19d5e71c6e', 'exit_code': '0', 'status': 'COMPLETED'}
Listing objects in bucket: gen3wf-qa-midrc-planx-pla-net-313 with prefix: ga4gh-tes/ba/5527daa7d08eba26c89e19d5e71c6e
Work Dir protocol: s3
{'process_name': 'dicom_to_png (2)', 'workDir': 'gen3wf-qa-midrc-planx-pla-net-313/ga4gh-tes/8b/4a50db61c97f2a226e86198fd68480', 'exit_code': '0', 'status': 'COMPLETED'}
Listing objects in bucket: gen3wf-qa-midrc-planx-pla-net-313 with prefix: ga4gh-tes/8b/4a50db61c97f2a226e86198fd68480
Work Dir protocol: s3
{'process_name': 'dicom_to_png (1)', 'wo

In [84]:
for process_name, s3_object in s3_contents.items():
    print(f"Process: {process_name}")
    if "Contents" in s3_object:
        for obj in s3_object["Contents"]:
            print(f"  - {obj['Key']}")
    else:
        print("  - No objects found")
    

Process: extract_metadata (2)
  - ga4gh-tes/b7/34a4f2e25237ce38d536961cf17527/
  - ga4gh-tes/b7/34a4f2e25237ce38d536961cf17527/.command.err
  - ga4gh-tes/b7/34a4f2e25237ce38d536961cf17527/.command.log
  - ga4gh-tes/b7/34a4f2e25237ce38d536961cf17527/.command.out
  - ga4gh-tes/b7/34a4f2e25237ce38d536961cf17527/.command.run
  - ga4gh-tes/b7/34a4f2e25237ce38d536961cf17527/.command.sh
  - ga4gh-tes/b7/34a4f2e25237ce38d536961cf17527/.exitcode
  - ga4gh-tes/b7/34a4f2e25237ce38d536961cf17527/dicom-metadata-1-008.dcm.csv
Process: extract_metadata (1)
  - ga4gh-tes/ba/5527daa7d08eba26c89e19d5e71c6e/
  - ga4gh-tes/ba/5527daa7d08eba26c89e19d5e71c6e/.command.err
  - ga4gh-tes/ba/5527daa7d08eba26c89e19d5e71c6e/.command.log
  - ga4gh-tes/ba/5527daa7d08eba26c89e19d5e71c6e/.command.out
  - ga4gh-tes/ba/5527daa7d08eba26c89e19d5e71c6e/.command.run
  - ga4gh-tes/ba/5527daa7d08eba26c89e19d5e71c6e/.command.sh
  - ga4gh-tes/ba/5527daa7d08eba26c89e19d5e71c6e/.exitcode
  - ga4gh-tes/ba/5527daa7d08eba26c89e19d5

In [85]:
s3_contents 

{'extract_metadata (2)': {'ResponseMetadata': {'HTTPStatusCode': 200,
   'HTTPHeaders': {'date': 'Fri, 25 Apr 2025 15:20:28 GMT',
    'content-type': 'application/xml',
    'content-length': '3046',
    'connection': 'keep-alive',
    'x-content-type-options': 'nosniff',
    'x-xss-protection': '1; mode=block',
    'strict-transport-security': 'max-age=63072000; includeSubdomains;',
    'access-control-allow-origin': '*',
    'access-control-allow-methods': 'GET, POST, OPTIONS, DELETE, PUT',
    'access-control-allow-headers': 'DNT,User-Agent,X-Requested-With,If-Modified-Since,Cache-Control,Content-Type,Range,Authorization,Cookie,X-CSRF-Token',
    'access-control-expose-headers': 'Content-Length,Content-Range',
    'set-cookie': 'service_releases=fence.production&fenceshib.production&sheepdog.production&indexd.production&peregrine.production&;Path=/;Max-Age=600;HttpOnly;Secure;SameSite=Lax, csrftoken=c95ee4815e5ab68fd9a0a98f1ce24d4919340.0002025-04-25T15:20:28+00:00;Path=/;Secure;Same

In [80]:
execution.process_executions

[<ProcessExecution: 01/48abd8>,
 <ProcessExecution: 17/7acd99>,
 <ProcessExecution: 49/dcdc18>,
 <ProcessExecution: f8/f3b2e3>]